In [72]:
from elasticsearch import Elasticsearch, helpers
import csv
import json 
# Connexion au cluster Elasticsearch
es = Elasticsearch(hosts="http://localhost:9200")

# Nom de l'index
index_name = "reviews"

# Mapping pour l'index 
mapping = {
    "mappings": {
        "_meta": {
            "created_by": "file-data-visualizer"
        },
        "properties": {
            "@timestamp": {"type": "date"},
            "Company": {"type": "keyword"},
            "Customer": {"type": "keyword"},
            "Date_experience": {"type": "date", "format": "yyyy-MM-dd HH:mm:ss"},
            "Date_reply": {"type": "date", "format": "yyyy-MM-dd HH:mm:ss"},
            "Date_review": {"type": "date", "format": "yyyy-MM-dd HH:mm:ss"},
            "Experience": {"type": "text"},
            "Language": {"type": "keyword"},
            "Number_review": {"type": "long"},
            "Rating": {"type": "long"},
            "Reply": {"type": "text"},
            "Status": {"type": "keyword"},
            "Title": {"type": "text"},
            "column1": {"type": "long"},
            "document_id": {"type": "keyword"}  # Ajout d'un champ pour l'ID du document
        }
    }
}

# Créer l'index avec le mapping (you can skip this if the index already exists)
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=mapping)

# Lecture du fichier CSV et correction des champs vides
with open("../data/avis_clients.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    documents = []
    for idx, row in enumerate(reader):
        if "" in row:
            row["document_id"] = str(idx)  # Ajout de l'ID du document
            for key in list(row.keys()):
                if key == "":
                    row.pop(key)  # Suppression des champs avec des clés vides
        documents.append(row)

# Insérer les documents en vrac dans Elasticsearch
if documents:
    try:
        response = helpers.bulk(es, documents, index=index_name)
        print("Documents insérés avec succès :", response)
    except helpers.BulkIndexError as e:
        print("Erreur lors de l'insertion des documents :")
        for err in e.errors:
            print("Erreur :", err)

# Vérification des documents insérés
result = es.search(index=index_name, size=5)  # Récupère les 5 premiers documents

# Afficher les résultats
for hit in result["hits"]["hits"]:
    print(hit["_source"])


Documents insérés avec succès : (2394, [])
{'Company': 'Younited Credit', 'Customer': 'M françois GUYOT', 'Number_review': '2', 'Language': 'FR', 'Title': 'SUPER SOCIETE DE CREDITS TRES REACTIVE…', 'Date_review': '2023-07-28 06:28:34', 'Reply': 'No Reply', 'Date_reply': '2023-07-28 06:28:34', 'Rating': '5', 'Status': 'Verified', 'Experience': 'SUPER SOCIETE DE CREDITS TRES REACTIVE…SUPER SOCIETE DE CREDITS TRES REACTIVE sans problêmes merci pour votre pret accorde avec le moins d administratif possible', 'Date_experience': '2023-07-20 00:00:00', 'document_id': '0'}
{'Company': 'Younited Credit', 'Customer': 'Les parisiens', 'Number_review': '2', 'Language': 'FR', 'Title': 'Simple et efficace !', 'Date_review': '2023-07-28 17:28:34', 'Reply': 'No Reply', 'Date_reply': '2023-07-28 17:28:34', 'Rating': '5', 'Status': 'Verified', 'Experience': 'Simple et efficace !', 'Date_experience': '2023-07-19 00:00:00', 'document_id': '1'}
{'Company': 'Younited Credit', 'Customer': 'STEPHANIE RENOUX',

In [65]:
response = es.search(index="reviews")
# Récupération du template
template = es.indices.get_mapping()



In [66]:
# recherche de tous les documents 
query = {
    "query": {
        "match_all": {}
    }
}

# Exécuter la requête
result = es.search(index="reviews", body=query)

# Afficher les résultats
for hit in result["hits"]["hits"]:
    print(hit["_source"])

    

{'Company': 'Younited Credit', 'Customer': 'M françois GUYOT', 'Number_review': '2', 'Language': 'FR', 'Title': 'SUPER SOCIETE DE CREDITS TRES REACTIVE…', 'Date_review': '2023-07-28 06:28:34', 'Reply': 'No Reply', 'Date_reply': '2023-07-28 06:28:34', 'Rating': '5', 'Status': 'Verified', 'Experience': 'SUPER SOCIETE DE CREDITS TRES REACTIVE…SUPER SOCIETE DE CREDITS TRES REACTIVE sans problêmes merci pour votre pret accorde avec le moins d administratif possible', 'Date_experience': '2023-07-20 00:00:00', 'document_id': '0'}
{'Company': 'Younited Credit', 'Customer': 'Les parisiens', 'Number_review': '2', 'Language': 'FR', 'Title': 'Simple et efficace !', 'Date_review': '2023-07-28 17:28:34', 'Reply': 'No Reply', 'Date_reply': '2023-07-28 17:28:34', 'Rating': '5', 'Status': 'Verified', 'Experience': 'Simple et efficace !', 'Date_experience': '2023-07-19 00:00:00', 'document_id': '1'}
{'Company': 'Younited Credit', 'Customer': 'STEPHANIE RENOUX', 'Number_review': '3', 'Language': 'FR', 'T

In [67]:
# recherche de documents contenant "bon produit"
query = {
    "query": {
        "match": {
            "Experience": "bon produit"
        }
    }
}

# Exécuter la requête
result = es.search(index="reviews", body=query)

# Afficher les résultats
for hit in result["hits"]["hits"]:
    print(hit["_source"])
    

{'Company': 'Boursorama Banque', 'Customer': 'FELIX', 'Number_review': '8', 'Language': 'MQ', 'Title': 'EXCELLENT  PRODUIT  FINANCIER', 'Date_review': '2022-11-03 00:00:00', 'Reply': 'No Reply', 'Date_reply': '2022-11-03 00:00:00', 'Rating': '5', 'Status': 'Invited', 'Experience': 'EXCELLENT  PRODUIT  FINANCIER', 'Date_experience': '2022-11-03 00:00:00', 'document_id': '1865'}
{'Company': 'Boursorama Banque', 'Customer': 'paulo dos reis', 'Number_review': '2', 'Language': 'FR', 'Title': 'nickel', 'Date_review': '2022-05-11 00:00:00', 'Reply': 'No Reply', 'Date_reply': '2022-05-11 00:00:00', 'Rating': '5', 'Status': 'Invited', 'Experience': 'nickelnickel, super produit', 'Date_experience': '2022-05-11 00:00:00', 'document_id': '2005'}
{'Company': 'Boursorama Banque', 'Customer': 'FELIX', 'Number_review': '8', 'Language': 'MQ', 'Title': 'EXCELLENT  PRODUIT  FINANCIER', 'Date_review': '2022-11-03 00:00:00', 'Reply': 'No Reply', 'Date_reply': '2022-11-03 00:00:00', 'Rating': '5', 'Status':

In [68]:
# Requête de recherche avec filtre de plage de documents avec 4<rating<5
query = {
    "query": {
        "range": {
            "Rating": {
                "gte": 4,
                "lte": 5
            }
        }
    }
}

# Exécuter la requête
result = es.search(index="reviews", body=query)

# Afficher les résultats
for hit in result["hits"]["hits"]:
    print(hit["_source"])

{'Company': 'Younited Credit', 'Customer': 'M françois GUYOT', 'Number_review': '2', 'Language': 'FR', 'Title': 'SUPER SOCIETE DE CREDITS TRES REACTIVE…', 'Date_review': '2023-07-28 06:28:34', 'Reply': 'No Reply', 'Date_reply': '2023-07-28 06:28:34', 'Rating': '5', 'Status': 'Verified', 'Experience': 'SUPER SOCIETE DE CREDITS TRES REACTIVE…SUPER SOCIETE DE CREDITS TRES REACTIVE sans problêmes merci pour votre pret accorde avec le moins d administratif possible', 'Date_experience': '2023-07-20 00:00:00', 'document_id': '0'}
{'Company': 'Younited Credit', 'Customer': 'Les parisiens', 'Number_review': '2', 'Language': 'FR', 'Title': 'Simple et efficace !', 'Date_review': '2023-07-28 17:28:34', 'Reply': 'No Reply', 'Date_reply': '2023-07-28 17:28:34', 'Rating': '5', 'Status': 'Verified', 'Experience': 'Simple et efficace !', 'Date_experience': '2023-07-19 00:00:00', 'document_id': '1'}
{'Company': 'Younited Credit', 'Customer': 'STEPHANIE RENOUX', 'Number_review': '3', 'Language': 'FR', 'T

In [69]:
# Requête de recherche avec l'opérateur or 
query = {
    "query": {
        "match": {
        "Experience": {
            "query": "simple rapide effice bien",
            "operator": "or"
            }
        }
    }
}

# Exécuter la requête
result = es.search(index="reviews", body=query)

# Afficher les résultats
for hit in result["hits"]["hits"]:
    print(hit["_source"])

{'Company': 'Younited Credit', 'Customer': 'ghislaine', 'Number_review': '2', 'Language': 'FR', 'Title': 'simple et bien', 'Date_review': '2018-04-18 00:00:00', 'Reply': 'No Reply', 'Date_reply': '2018-04-18 00:00:00', 'Rating': '5', 'Status': 'Invited', 'Experience': 'simple et bien', 'Date_experience': '2018-04-18 00:00:00', 'document_id': '527'}
{'Company': 'Younited Credit', 'Customer': 'ghislaine', 'Number_review': '2', 'Language': 'FR', 'Title': 'simple et bien', 'Date_review': '2018-04-18 00:00:00', 'Reply': 'No Reply', 'Date_reply': '2018-04-18 00:00:00', 'Rating': '5', 'Status': 'Invited', 'Experience': 'simple et bien', 'Date_experience': '2018-04-18 00:00:00', 'document_id': '527'}
{'Company': 'Younited Credit', 'Customer': 'ghislaine', 'Number_review': '2', 'Language': 'FR', 'Title': 'simple et bien', 'Date_review': '2018-04-18 00:00:00', 'Reply': 'No Reply', 'Date_reply': '2018-04-18 00:00:00', 'Rating': '5', 'Status': 'Invited', 'Experience': 'simple et bien', 'Date_exper

In [70]:
from elasticsearch import Elasticsearch

# Connexion au cluster Elasticsearch
es = Elasticsearch(hosts="http://localhost:9200")

# Requête d'agrégation pour obtenir des statistiques sur le champ "Rating"
query = {
    "size": 0,
    "aggs": {
        "value_count": {
            "value_count": {
                "field": "Rating"
            }
        },
        "rating_stats": {
            "stats": {
                "field": "Rating"
            }
        },
        "extended_ratings_stats": {
            "extended_stats": {
                "field": "Rating"
            }
        }
    }
}

# Exécuter la requête
result = es.search(index="reviews", body=query)

# Afficher les résultats de l'agrégation
aggregations = result.get("aggregations", {})
value_count = aggregations.get("value_count", {})
rating_stats = aggregations.get("rating_stats", {})
extended_ratings_stats = aggregations.get("extended_ratings_stats", {})

print("Nombre total de valeurs de rating:", value_count.get("value", 0))
print("Statistiques de rating:")
print("    Minimum:", rating_stats.get("min", 0))
print("    Maximum:", rating_stats.get("max", 0))
print("    Moyenne:", rating_stats.get("avg", 0))
print("    Somme:", rating_stats.get("sum", 0))
print("    Écart-type:", extended_ratings_stats.get("std_deviation", 0))


Nombre total de valeurs de rating: 7182
Statistiques de rating:
    Minimum: 1.0
    Maximum: 5.0
    Moyenne: 4.7180451127819545
    Somme: 33885.0
    Écart-type: 0.7912112398310651


In [74]:
# Requête de recherche avec agrégation pour le nombre total de docs pour chaque entreprise
query = {
    "size": 0,
    "aggs": {
        "companies": {
            "terms": {
                "field": "Company",
                "size": 10
            }
        }
    }
}

# Exécuter la requête
result = es.search(index="reviews", body=query)

# Afficher les résultats de l'agrégation
for bucket in result["aggregations"]["companies"]["buckets"]:
    print(bucket["key"], ":", bucket["doc_count"])

Orange Bank : 3048
Younited Credit : 2776
Boursorama Banque : 1524
Anytime : 948
Cofidis : 948
Floabank : 332
